In [2]:
import torch
import dqc
import dqc.xc
import dqc.utils

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\theta}} = 
 \int \rho(\vec{r}) \frac{\partial \epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}d\vec{r} 
$$
so we just can put $\frac{\partial \epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}$ instead of $\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})$ to DQC calculation.

In [3]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [4]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [5]:
mol = dqc.Mol(moldesc="H -1 0 0; H 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-0.4645, dtype=torch.float64, grad_fn=<AddBackward0>)


In [6]:
dm = qc._dm.detach().clone() # density matrix
dm

tensor([[ 4.3848e-31, -1.4465e-16, -6.4918e-31, -9.2522e-16],
        [-1.4465e-16,  4.7720e-02,  2.1416e-16,  3.0523e-01],
        [-6.4918e-31,  2.1416e-16,  9.6112e-31,  1.3698e-15],
        [-9.2522e-16,  3.0523e-01,  1.3698e-15,  1.9523e+00]],
       dtype=torch.float64)

In [14]:
from typing import List, Optional, Union, overload, Tuple, Type
from dqc.utils.datastruct import AtomCGTOBasis, ValGrad, SpinParam, DensityFitInfo

# calculate partitial derivative of (full) energy with respect to parameters of xc-functional
def get_e_xc_derivative(hamiltonian, dm: Union[torch.Tensor, SpinParam[torch.Tensor]], number_of_parameter) -> torch.Tensor:
    densinfo = SpinParam.apply_fcn(
        lambda dm_: hamiltonian._dm2densinfo(dm_), dm)  # (spin) value: (*BD, nr)
    edens_derivative = hamiltonian.xc.get_edensityxc_derivative(densinfo, number_of_parameter)  # (*BD, nr)
    return torch.sum(hamiltonian.grid.get_dvolume() * edens_derivative, dim=-1)

In [8]:
get_e_xc_derivative(qc.get_system().get_hamiltonian(), dm, 0)

tensor(0.0711, dtype=torch.float64, grad_fn=<SumBackward1>)

In [9]:
get_e_xc_derivative(qc.get_system().get_hamiltonian(), dm, 1)

tensor(2.0000, dtype=torch.float64, grad_fn=<SumBackward1>)